# Project: Summarization App Using LangChain and OpenAI
This is part of my **"Learn LangChain, Pinecone & OpenAI: Build Next-Gen LLM Apps"** course.

https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/?referralCode=4B17E3BD4CBBEA3B8321

This notebook uses **the latest versions** of the OpenAI and LangChain libraries.

In [ ]:
pip install -q -r ./requirements.txt

Download [requirements.txt](https://drive.google.com/file/d/1UpURYL9kqjXfe9J8o-_Dq5KJTbQpzMef/view?usp=sharing)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### A) Basic Prompt

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [3]:
text= r"""
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')



In [4]:
llm.get_num_tokens(text)

238

In [5]:
summary_output = llm.invoke(messages)

In [6]:
print(summary_output.content)

Mojo is a new programming language that combines the usability of Python with the performance of C. It aims to bridge the gap between research and production in the field of AI by offering portable code that is faster than C and seamlessly integrates with the Python ecosystem. The development of Mojo was driven by the need to simplify programming across the entire ML/AI infrastructure and improve the experience of writing MLIR. Additionally, Mojo recognizes the importance of host CPUs as accelerators and provides support for operations that specialized accelerators may not handle.


### Summarizing Using Prompt Templates

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [8]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [9]:
llm.get_num_tokens(prompt.format(text=text, language='English'))

259

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.invoke({'text': text, 'language':'hindi'})

In [11]:
print(summary)

{'text': 'Mojo एक नया प्रोग्रामिंग भाषा है जो पायथन की उपयोगिता को सी की प्रदर्शन के साथ मिलाकर AI हार्डवेयर की अद्वितीय प्रोग्रामबिलिटी और AI मॉडल की विस्तारयोग्यता को खोलता है। Mojo एक नई प्रोग्रामिंग भाषा है जो शोध और उत्पादन के बीच की खाई को पायथन की सर्वश्रेष्ठता के साथ सिस्टम प्रोग्रामिंग और मेटाप्रोग्रामिंग को मिलाकर बनाती है। Mojo के साथ, आप C से तेज़ पोर्टेबल कोड लिख सकते हैं और पायथन इकोसिस्टम के साथ सहजता से इंटर-ऑप कर सकते हैं।', 'language': 'hindi'}


### Summarizing using SuffDocumentChain

In [12]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document


In [13]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [14]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)
prompt

PromptTemplate(input_variables=['text'], template='Write a concise and short summary of the following text.\nTEXT: `{text}`\n')

In [15]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.invoke(docs)

In [16]:
# output_summary is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output_summary['output_text'])

The speaker, Steve Jobs, shares three stories from his life during a commencement speech. The first story is about dropping out of college and how it led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about getting fired from Apple and how it allowed him to start over and eventually create successful companies like NeXT and Pixar. The third story is about facing death when he was diagnosed with cancer and how it made him realize the importance of following his heart and not wasting time. He ends the speech by encouraging the graduates to stay hungry and stay foolish.


### Summarizing Large Documents Using map_reduce

In [17]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [19]:
llm.get_num_tokens(text)

2653

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [21]:
len(chunks)

2

In [22]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.invoke(chunks)

In [23]:
# output_summary is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output_summary['output_text'])

Steve Jobs shares three stories from his life, including dropping out of college and how it influenced the design of the Macintosh computer, getting fired from Apple and finding success in new ventures, and his experience with cancer and the importance of living each day fully. He encourages the audience to follow their passions, not settle, and remember that life is short. The speaker also reflects on the inevitability of death, urges the audience to live their own lives, and mentions The Whole Earth Catalog's message of staying hungry and foolish.


In [24]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [25]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### map_reduce wich Custom Prompts

In [26]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [27]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [28]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
output = summary_chain.invoke(chunks)

In [29]:
# output is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output['output_text'])

Title: Steve Jobs' Speech on Life Lessons and Following Your Passion

Introduction:
In this speech, Steve Jobs shares three stories from his life, highlighting the importance of following one's passion and not settling for anything less. He discusses how his experiences with dropping out of college, getting fired from Apple, and battling cancer shaped his perspective on life.

Key Points:
- Story 1: Jobs dropped out of college and took a calligraphy class, which later influenced the design of the Macintosh computer.
- Story 2: Getting fired from Apple led Jobs to start new ventures and find success.
- Story 3: Jobs' experience with cancer reminded him of the importance of living each day to the fullest.
- Jobs encourages the audience to follow their passions and not be influenced by others' opinions.
- The text discusses the inevitability of death and emphasizes the importance of living one's own life.
- Following one's heart and intuition is emphasized.
- The text mentions The Whole E

### Summarizing Using the refine Chain

In [30]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader

In [31]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
# pip install -q pdf2image

In [ ]:
# pip install -q pdfminer

In [32]:
# loader = UnstructuredPDFLoader('files/attention_is_all_you_need.pdf')
loader = PyPDFLoader('files/attention_is_all_you_need.pdf')
data = loader.load()

In [ ]:
# print(data[0].page_content)

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [34]:
len(chunks)

15

In [35]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [36]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.invoke(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine trans

In [38]:
# print(output_summary)

### refine With Custom Prompts

In [39]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)


In [40]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.invoke(chunks)

In [ ]:
# print(output_summary)

In [41]:
# displaying the refined summary
print(output_summary['output_text'])

The Transformer model is a groundbreaking network architecture in the field of natural language processing. It incorporates multi-head attention, position-wise feed-forward networks, embeddings, and positional encoding to achieve impressive results in various tasks. The model's reliance on attention mechanisms and the ability to learn long-range dependencies make it a powerful tool for sequence transduction tasks. Additionally, the Transformer model offers advantages in terms of computational complexity, parallelizability, and improved efficiency. It has been trained on large datasets and utilizes hardware acceleration for efficient training. The model's performance has been evaluated on machine translation tasks, where it outperforms previous state-of-the-art models with significantly lower training costs. Furthermore, the Transformer model has been shown to generalize well to English constituency parsing, demonstrating its potential for tackling diverse natural language processing ta

### Summarizing Using LangChain Agents

In [42]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [43]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [44]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [45]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [46]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

/home/dda/projects/python/AI_Notebooks_OpenAI_LangChain_Copilot_Gemini/venv312/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [47]:
output = agent_executor.invoke('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should use Wikipedia to find a short summary of George Washington.
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, politician and statesman who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted and ratified the Constitution of the United States and established the U.S. federal government. Washington has thus been known as the "Father of the Nation".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. He subsequently received military training and was assigned com

In [48]:
print(output['input'])

Can you please provide a short summary of George Washington?


In [49]:
print(output['output'])

George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, politician, and statesman who served as the first president of the United States from 1789 to 1797. He played a crucial role in the American Revolutionary War and was instrumental in the drafting and ratification of the United States Constitution. Washington is often referred to as the "Father of the Nation" and is considered one of the greatest presidents in American history.
